In [39]:
import sqlite3
import pandas as pd
import numpy as np
from datetime import datetime
from pytz import timezone
import statistics

In [75]:
# read in data
data = pd.read_csv('Seizures.csv')

data['pre-max'] = np.nan
data['pre-min'] = np.nan
data['pre-mean'] = np.nan
data['pre-stdev'] = np.nan
data['pre-n'] = np.nan
data['post-max'] = np.nan
data['post-min'] = np.nan
data['post-mean'] = np.nan
data['post-stdev'] = np.nan
data['post-n'] = np.nan

conn304 = sqlite3.connect("E:\\eeg\\data\\AM304eeg.db")
conn305 = sqlite3.connect("E:\\eeg\\data\\AM305eeg.db")

In [76]:
data

,animal,date,stim,start,end,score,pre-max,pre-min,pre-mean,pre-stdev,pre-n,post-max,post-min,post-mean,post-stdev,post-n
0,AM304,11/1/2018,1.0,15:29:00,15:31:00,3+,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AM304,11/2/2018,2.0,14:39:30,14:41:30,3+,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AM304,11/3/2018,3.0,14:10:32,14:12:32,2-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AM304,11/4/2018,4.0,13:27:00,13:29:00,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AM304,11/5/2018,5.0,15:49:00,15:51:00,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,AM304,11/6/2018,6.0,13:17:00,13:19:00,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,AM304,11/6/2018,NaN,13:23:32,13:25:32,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,AM304,11/7/2018,7.0,14:49:15,14:51:15,2-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,AM304,11/8/2018,8.0,13:01:00,13:03:00,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,AM304,11/8/2018,NaN,13:08:02,13:08:30,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
def get_temp_stats(animal, date, start_time, end_time):
    

    if animal == "AM304":
        conn = conn304
    elif animal == "AM305":
        conn = conn305
    
    
    cur = conn.cursor()
    #print("select * from eeg limit 100 where time between {} and {};".format(start_epoch, end_epoch))
    cur.execute("select * from temp where time between {} and {};".format(start_time, end_time))
    sqlresult = cur.fetchall()
    cur.close()
    data = pd.DataFrame(sqlresult, columns=['Time', 'Temp'])
    temps = [float(x) for x in data['Temp'] if str(x) != 'nan']
    stats = {}
    stats['max'] = max(temps)
    stats['min'] = min(temps)
    stats['mean'] = statistics.mean(temps)
    stats['stdev'] = statistics.stdev(temps)
    stats['n'] = len(temps)
    
    return stats

In [49]:
def get_pre_post_stats(animal, date, start_time, end_time):
    tz = timezone('US/Central')
    start_time = f'{date} {start_time}'
    end_time = f'{date} {end_time}'
    
    print(tz.localize(datetime.strptime(start_time, "%m/%d/%Y %H:%M:%S")))
    
    pre_start_epoch = tz.localize(datetime.strptime(start_time, "%m/%d/%Y %H:%M:%S")).timestamp() - 600 #10 minutes
    pre_end_epoch = tz.localize(datetime.strptime(start_time, "%m/%d/%Y %H:%M:%S")).timestamp()
    pre_stats = get_temp_stats(animal, date, pre_start_epoch, pre_end_epoch)
    
    post_start_epoch = tz.localize(datetime.strptime(end_time, "%m/%d/%Y %H:%M:%S")).timestamp()
    post_end_epoch = tz.localize(datetime.strptime(end_time, "%m/%d/%Y %H:%M:%S")).timestamp() + 3600 #60 minutes
    post_stats = get_temp_stats(animal, date, post_start_epoch, post_end_epoch)
    
    return pre_stats, post_stats

In [77]:
for i in range(len(data)):
    animal = data['animal'][i]
    date = data['date'][i]
    start = data['start'][i]
    end = data['end'][i]
    pre_stats, post_stats = get_pre_post_stats(animal, date, start, end)
    print(pre_stats)
    print(post_stats)
    
    data.at[i, 'pre-max'] = pre_stats['max']
    data.at[i, 'pre-min'] = pre_stats['min']
    data.at[i, 'pre-mean'] = pre_stats['mean']
    data.at[i, 'pre-stdev'] = pre_stats['stdev']
    data.at[i, 'pre-n'] = pre_stats['n']
    data.at[i, 'post-max'] = post_stats['max']
    data.at[i, 'post-min'] = post_stats['min']
    data.at[i, 'post-mean'] = post_stats['mean']
    data.at[i, 'post-stdev'] = post_stats['stdev']
    data.at[i, 'post-n'] = post_stats['n']

2018-11-01 15:29:00-05:00
{'max': 37.679, 'min': 36.974000000000004, 'mean': 37.42749750415973, 'stdev': 0.12613640927091754, 'n': 601}
{'max': 38.369, 'min': 37.035, 'mean': 37.71809182249512, 'stdev': 0.33851085472840525, 'n': 3583}
2018-11-02 14:39:30-05:00
{'max': 37.121, 'min': 36.675, 'mean': 36.89627441077441, 'stdev': 0.06435012092038694, 'n': 594}
{'max': 38.598, 'min': 35.787, 'mean': 37.89349564433764, 'stdev': 0.5408132320674296, 'n': 3329}
2018-11-03 14:10:32-05:00
{'max': 37.035, 'min': 36.431, 'mean': 36.79561871750433, 'stdev': 0.09369476259607945, 'n': 577}
{'max': 38.815, 'min': 36.813, 'mean': 38.165131088227106, 'stdev': 0.4685318178963391, 'n': 3593}
2018-11-04 13:27:00-06:00
{'max': 36.77, 'min': 32.274, 'mean': 36.524699481865284, 'stdev': 0.2765192518546058, 'n': 1158}
{'max': 38.232, 'min': 36.468, 'mean': 37.372746675531914, 'stdev': 0.5025264499572656, 'n': 6016}
2018-11-05 15:49:00-06:00
{'max': 37.661, 'min': 37.118, 'mean': 37.37841652613828, 'stdev': 0.09

In [80]:
data
data.to_csv('temperature stats.csv')